# Capstone Project

## Introduction:

Toronto is the most multicultural city in Canada. It provides lot of business oppourtunities and business friendly environment. It is a global hub of business and commerce. The market is highly competitive here.So, any new business venture needs to be analysed carefully. However exciting it is to start a new business, there's a huge amount of risk involved in setting up one. The insights derived from analysis will give us good and breif understanding of the business environment. This will help in reduction of risk. In this project, we are going to find the most suitable location to open a new coffee business that shall yeild profit.

## Business Problem

The aim of this Capstone project is to find the most suitable location for the entrepreneur to open a new coffee shop in Toronto, Canada. By using foursquareapi and other supporting libraries, this project aims to provide solutions to the question: In Toronto, if an entrepreneur wants to open a Coffee shop, where should he/she would consider opening it?

## Target Audience

Entrepreneur who wants to set up a coffee shop in the location that will profit him/her.

# Data:

### The following data sources will be needed to generate the required outcome:

1. List of postal codes of Canada (from wikipedia page)
2. Latitude and Longitude of these postal code (toronto_loc.csv)


Source data needs to taken from: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada

# Methodology :

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [4]:
soup = BeautifulSoup(source,'lxml')

In [6]:
Table = soup.find('table', class_ = "wikitable sortable")

In [7]:
dt = ""
for tr in Table.find_all('tr'):
    row1 = ""
    for td in tr.find_all('td'):
        row1 = row1 + "," + td.text
    dt = dt + row1[1:] 
print(dt)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Not assigned
M8A,Not assigned,Not assigned
M9A,Downtown Toronto,Queen's Park
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M5C,Downtown Toronto,St. James 

In [8]:
csv_file = open("toronto.csv","wb")
csv_file.write(bytes(dt,encoding="ascii",errors="ignore"))

8708

In [9]:
df = pd.read_csv('toronto.csv',header = None)
df.columns = ['PostalCode','Borough','Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
indexes = df[df['Borough'] == "Not assigned"].index
df.drop(indexes, inplace = True)

In [11]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [12]:
df.loc[df['Neighborhood'] == "Not assigned", "Neighborhood"] = df["Borough"]
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [13]:
res = df.groupby(['PostalCode','Borough'], sort = False).agg(','.join)
res

,,Neighborhood
PostalCode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park
M9A,Downtown Toronto,Queen's Park
M1B,Scarborough,"Rouge,Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens,Parkview Hill"


In [14]:
df1 = res.reset_index()
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [15]:
df1.shape

(103, 3)

In [16]:
!wget -q -O 'toronto_loc.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [17]:
df_loc = pd.read_csv('toronto_loc.csv')
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_loc.columns = ['PostalCode','Latitude','Longitude']

In [19]:
df_loc.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
Toronto_lat_lng = pd.merge(df1,df_loc, on="PostalCode")
Toronto_lat_lng.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [21]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [22]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_lat_lng['Latitude'], Toronto_lat_lng['Longitude'], Toronto_lat_lng['Borough'], Toronto_lat_lng['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [23]:
CLIENT_ID = '3RXJEAVZHT3XCUZEIVAMG3STOQ0PQXSQHKR4ABJAX34FTTWR' # your Foursquare ID
CLIENT_SECRET = 'HZ0LJ1ADCMST42CNN4U2HGJ5Q0ACVBDJZWN5X0SQTOE3EUAA' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3RXJEAVZHT3XCUZEIVAMG3STOQ0PQXSQHKR4ABJAX34FTTWR
CLIENT_SECRET:HZ0LJ1ADCMST42CNN4U2HGJ5Q0ACVBDJZWN5X0SQTOE3EUAA


In [24]:
LIMIT=100

In [25]:
#search_query = 'Coffee'
search_query = 'Office'
#search_query = 'BookStore'
#search_query = 'University'
B = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, 500, LIMIT)
B

'https://api.foursquare.com/v2/venues/search?client_id=3RXJEAVZHT3XCUZEIVAMG3STOQ0PQXSQHKR4ABJAX34FTTWR&client_secret=HZ0LJ1ADCMST42CNN4U2HGJ5Q0ACVBDJZWN5X0SQTOE3EUAA&ll=43.653963,-79.387207&v=20180604&query=Office&radius=500&limit=100'

In [26]:
pj = requests.get(B).json()
pj

{'meta': {'code': 200, 'requestId': '5df12afa47e0d64ab922f83d'},
 'response': {'venues': [{'id': '599295fa345cbe217d950320',
    'name': 'Law Office of Fese N. Elango',
    'location': {'address': '439 University Ave, 5th Floor',
     'lat': 43.6544588,
     'lng': -79.3873495,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.6544588,
       'lng': -79.3873495}],
     'distance': 56,
     'postalCode': 'M5G 1Y8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['439 University Ave, 5th Floor',
      'Toronto ON M5G 1Y8',
      'Canada']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b3f',
      'name': 'Lawyer',
      'pluralName': 'Lawyers',
      'shortName': 'Lawyer',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/lawschool_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1576086299',
    'hasPerk': False},
   {'id': '4b4de663f964a52089da26e3',
    'n

In [27]:
ven = pj['response']['venues']
type(ven)

list

In [28]:
from pandas.io.json import json_normalize
dataframe = json_normalize(ven)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52f2ab2ebcbc57f1066b8b3f', 'name': 'L...",False,599295fa345cbe217d950320,"439 University Ave, 5th Floor",CA,Toronto,Canada,NaN,56,"[439 University Ave, 5th Floor, Toronto ON M5G...","[{'label': 'display', 'lat': 43.6544588, 'lng'...",43.654459,-79.387349,M5G 1Y8,ON,Law Office of Fese N. Elango,v-1576086299,NaN
1,"[{'id': '4bf58dd8d48988d177941735', 'name': 'D...",False,4b4de663f964a52089da26e3,Atrium on Bay,CA,Toronto,Canada,Dundas,379,"[Atrium on Bay (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65610458190151...",43.656105,-79.383540,NaN,ON,The Doctors Office,v-1576086299,NaN
2,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4e2976098877f9158a7f2912,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,380,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65736296792668...",43.657363,-79.387748,NaN,ON,Urology Office,v-1576086299,NaN
3,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4b73067af964a5208e962de3,"290 Yonge St, 6th Fl",CA,Toronto,Canada,NaN,518,"[290 Yonge St, 6th Fl, Toronto ON M5B 2C3, Can...","[{'label': 'display', 'lat': 43.65510784096450...",43.655108,-79.380968,M5B 2C3,ON,Sears Canada Head Office,v-1576086299,NaN
4,"[{'id': '52f2ab2ebcbc57f1066b8b3f', 'name': 'L...",False,546516f9498e9ebfe242f48f,393 University Ave #2000,CA,Toronto,Canada,NaN,3,"[393 University Ave #2000, Toronto ON M5G 1E6,...","[{'label': 'display', 'lat': 43.65393168261403...",43.653932,-79.387228,M5G 1E6,ON,Geary B Shorser Law Office,v-1576086299,NaN


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,  categoryIds=''):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        
        # return only relevant information for each nearby venue
        for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name,
                        lat,
                        lng,
                        v['name'],
                        v['location']['lat'],
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names=Toronto_lat_lng['Neighborhood'],
                                   latitudes=Toronto_lat_lng['Latitude'],
                                   longitudes=Toronto_lat_lng['Longitude']
                                  )

In [31]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Cassandra Public School,43.748291,-79.328889,School
3,Parkwoods,43.753259,-79.329656,A&W Canada,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Food Basics,43.760865,-79.326015,Supermarket


In [32]:
toronto_venues_coffee = getNearbyVenues(names=Toronto_lat_lng['Neighborhood'],latitudes=Toronto_lat_lng['Latitude'],longitudes=Toronto_lat_lng['Longitude'], radius=1000, 
                                        categoryIds='4bf58dd8d48988d1e0931735')
toronto_venues_coffee.head() 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760670,-79.326589,Coffee Shop
1,Parkwoods,43.753259,-79.329656,La Notre,43.760704,-79.325396,Coffee Shop
2,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.723399,-79.302647,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.727082,-79.327143,Coffee Shop


In [35]:
toronto_venues_coffee.shape

(1580, 7)

In [36]:
toronto_venues_office = getNearbyVenues(names=Toronto_lat_lng['Neighborhood'],latitudes=Toronto_lat_lng['Latitude'],longitudes=Toronto_lat_lng['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d125941735')
toronto_venues_office.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Here,43.745004,-79.328979,Tech Startup
1,Harbourfront,43.654260,-79.360636,Equinix TR2,43.651083,-79.362553,Tech Startup
2,Harbourfront,43.654260,-79.360636,StackAdapt,43.651215,-79.369269,Tech Startup
3,Harbourfront,43.654260,-79.360636,Acquia,43.652475,-79.371025,Tech Startup
4,Harbourfront,43.654260,-79.360636,Drop HQ,43.649563,-79.371271,Tech Startup


In [37]:
toronto_venues_office.shape

(1006, 7)

In [38]:
def addToMap(df,color,existingMap):
    # add markers to map
    for lat, lng, venue, category in zip(df['Venue Latitude'], df['Venue Longitude'], df['Venue'], df['Venue Category']):
        label = '{}, {}'.format(venue, category)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            parse_html=False).add_to(existingMap)

In [40]:
map_office = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(toronto_venues_office,"yellow",map_office)
map_office

In [41]:
toronto_venues_bookstore = getNearbyVenues(names=Toronto_lat_lng['Neighborhood'],latitudes=Toronto_lat_lng['Latitude'],longitudes=Toronto_lat_lng['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d114951735')
toronto_venues_bookstore.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.654260,-79.360636,George Brown Book Store,43.650984,-79.369630,Bookstore
1,Harbourfront,43.654260,-79.360636,The Beguiling - Library Services Office,43.665188,-79.368110,Bookstore
2,Harbourfront,43.654260,-79.360636,D.E Lake Bookstore,43.649892,-79.370458,Bookstore
3,Harbourfront,43.654260,-79.360636,International News,43.665516,-79.368490,Bookstore
4,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Pauline Books & Media,43.712498,-79.454806,Bookstore


In [42]:
toronto_venues_bookstore.shape

(476, 7)

In [43]:
map_bookstore = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(toronto_venues_bookstore,"green",map_bookstore)
map_bookstore

In [44]:
toronto_venues_univ = getNearbyVenues(names=Toronto_lat_lng['Neighborhood'],latitudes=Toronto_lat_lng['Latitude'],longitudes=Toronto_lat_lng['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
toronto_venues_univ.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Queen's Park,43.662301,-79.389494,C. David Naylor Building,43.660310,-79.391205,University
1,Queen's Park,43.662301,-79.389494,J. M. Kelly Library,43.665235,-79.389639,College Library
2,Queen's Park,43.662301,-79.389494,Medical Sciences Building,43.661362,-79.393900,Medical School
3,Queen's Park,43.662301,-79.389494,Rotman School of Management,43.664797,-79.398576,College Academic Building
4,Queen's Park,43.662301,-79.389494,University of Toronto,43.662493,-79.395220,University


In [45]:
toronto_venues_univ.shape

(549, 7)

In [46]:
map_univ = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(toronto_venues_univ,"red",map_univ)
map_univ

In [47]:
def addColumn(newdf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in newdf['Neighborhood']:
        try:
            newdf.loc[newdf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            newdf.loc[newdf['Neighborhood'] == n,columnTitle] = 0

In [48]:
df2 = toronto_venues.copy()

In [49]:
df2.drop(df2.columns[[3,4,5,6]],axis=1,inplace=True)
df2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Parkwoods,43.753259,-79.329656
1,Parkwoods,43.753259,-79.329656
2,Parkwoods,43.753259,-79.329656
3,Parkwoods,43.753259,-79.329656
4,Parkwoods,43.753259,-79.329656


In [50]:
df_data = df2.copy()
df_data.rename(columns={'Neighborhood Latitude':'Latitude','Neighborhood Longitude':'Longitude'}, inplace=True)
addColumn(df_data, 'Coffee', toronto_venues_coffee)
addColumn(df_data, 'Office', toronto_venues_office)
addColumn(df_data, 'BookStore', toronto_venues_bookstore)
addColumn(df_data, 'University', toronto_venues_univ)
df_data.head()

,Neighborhood,Latitude,Longitude,Coffee,Office,BookStore,University
0,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
1,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
2,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
3,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
4,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0


In [52]:
df_data

,Neighborhood,Latitude,Longitude,Coffee,Office,BookStore,University
0,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
1,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
2,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
3,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
4,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
5,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
6,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
7,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
8,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0
9,Parkwoods,43.753259,-79.329656,2.0,1.0,0.0,0.0


In [53]:
weight_coffee = -1

# positive weight, because bookstore customers are good customers
weight_bookstore = 1
# positive weight, because univ students are good customers

weight_univ = 1.5

# positive weight, because office guys are good customers
weight_office = 2

In [54]:
df_weighted = df_data[['Neighborhood']].copy()

In [55]:
df_weighted['Score'] = df_data['Coffee'] * weight_coffee + df_data['Office'] * weight_office + df_data['BookStore'] * weight_bookstore + df_data['University'] * weight_univ
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted.head()

,Neighborhood,Score
2627,"Adelaide,King,Richmond",145.5
2624,"Adelaide,King,Richmond",145.5
2626,"Adelaide,King,Richmond",145.5
2628,"Adelaide,King,Richmond",145.5
2629,"Adelaide,King,Richmond",145.5


In [56]:
df2[df2['Neighborhood'] == 'Adelaide,King,Richmond']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
2601,"Adelaide,King,Richmond",43.650571,-79.384568
2602,"Adelaide,King,Richmond",43.650571,-79.384568
2603,"Adelaide,King,Richmond",43.650571,-79.384568
2604,"Adelaide,King,Richmond",43.650571,-79.384568
2605,"Adelaide,King,Richmond",43.650571,-79.384568
2606,"Adelaide,King,Richmond",43.650571,-79.384568
2607,"Adelaide,King,Richmond",43.650571,-79.384568
2608,"Adelaide,King,Richmond",43.650571,-79.384568
2609,"Adelaide,King,Richmond",43.650571,-79.384568
2610,"Adelaide,King,Richmond",43.650571,-79.384568


In [57]:
map_result = folium.Map(location=[latitude, longitude], zoom_start=15)

best_loc = df2[df2['Neighborhood'] == 'Adelaide,King,Richmond']

for lat, lng, local in zip(best_loc['Neighborhood Latitude'], best_loc['Neighborhood Longitude'], best_loc['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_result) 

addToMap(toronto_venues_coffee[toronto_venues_coffee['Neighborhood'] == 'Adelaide,King,Richmond'], 'red', map_result)
addToMap(toronto_venues_bookstore[toronto_venues_bookstore['Neighborhood'] == 'Adelaide,King,Richmond'], 'green', map_result)
addToMap(toronto_venues_univ[toronto_venues_univ['Neighborhood'] == 'Adelaide,King,Richmond'], 'gold', map_result)
addToMap(toronto_venues_office[toronto_venues_office['Neighborhood'] == 'Adelaide,King,Richmond'], 'fuchsia', map_result)

map_result

### Adelaide,King,Richmond is the best location to open up Coffee shop. Its pretty clear that Christie is the bad choice to open up coffee shop

# Conclusion:

#### In this project, we have gone through the following aspects:
    
1. Identifying the business problem.
2. Specifying the data required.
3. Extracting and cleaning the data.
4. Performing necessary actions to find out the best location to set up coffee shop.
5. Providing recommendation to the stakeholder.

# References:

1. List of Postal Code (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)
2. Foursquare Developer Documentation (https://developer.foursquare.com/docs)